In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Code')

from utility_functions import *
from weight_matrix import *
from parameters import *
from sim_network_EI import sim_network_Perturb
from scipy.special import lambertw, exp1, expi, hyp2f1, gamma, gammaincc
from scipy import optimize, integrate
import mpmath as mp
from natsort import natsorted
from glob import glob
from phi import *
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# from hmmlearn import hmm


In [2]:
E = np.arange(0, 2, 0.001)
g = np.arange(0, 1, 0.001)
J = 8.0
theta = np.array([0,1,2,3,4,5])

In [3]:
# Phase plot
E_plot_phase0 = np.zeros((len(g), len(theta)))
E_plot_phase1 = np.zeros((len(g), len(theta)))

for i in range(len(theta)):

    lambert0 = lambertw(-np.exp(theta[i] + 1 - J*(1-g)), 0)
    lambert1 = lambertw(-np.exp(theta[i] + 1 - J*(1-g)), -1)
    E_plot_phase0[:, i] = J*(1-g) - (1 - lambert0) * (1 + np.exp(J*(1-g) - 1 - theta[i] + lambert0))
    E_plot_phase1[:, i] = J*(1-g) - (1 - lambert1) * (1 + np.exp(J*(1-g) - 1 - theta[i] + lambert1))

    try:
        idxg = np.where(np.abs(J*(1-g) - theta[i] - 2) < 10e-4)[0][0]
        E_plot_phase0[idxg:, i] = np.nan
        E_plot_phase1[idxg:, i] = np.nan
    except:
        continue

/var/folders/7x/_g6cnzvd605glvq83cdzyh1r0000gn/T/ipykernel_43014/2056196615.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  E_plot_phase0[:, i] = J*(1-g) - (1 - lambert0) * (1 + np.exp(J*(1-g) - 1 - theta[i] + lambert0))
/var/folders/7x/_g6cnzvd605glvq83cdzyh1r0000gn/T/ipykernel_43014/2056196615.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  E_plot_phase1[:, i] = J*(1-g) - (1 - lambert1) * (1 + np.exp(J*(1-g) - 1 - theta[i] + lambert1))


In [4]:
E_fixed = -2.0
J_var = np.arange(1, 10, 0.001)

gg, JJ = np.meshgrid(g, J_var)

lambert0 = np.real(lambertw(-np.exp(theta[1] + 1 - JJ*(1-gg)), 0))
lambert1 = np.real(lambertw(-np.exp(theta[1] + 1 - JJ*(1-gg)), -1))
fun0 = E_fixed - ( JJ*(1-gg) - (1 - lambert0) * (1 + np.exp(JJ*(1-gg) - 1 - theta[1] + lambert0)) )
fun1 = E_fixed - ( JJ*(1-gg) - (1 - lambert1) * (1 + np.exp(JJ*(1-gg) - 1 - theta[1] + lambert1)) )

idx0 = np.where(np.abs(fun0) < 10e-4)
idx1 = np.where(np.abs(fun1) < 10e-4)

In [5]:
parameters = network_params()
parameters.dt = 0.01
parameters.N = 100
parameters.nE = 0.8  # Fraction of excitatory neurons
parameters.nI = 0.2  # Fraction of inhibotory neurons
parameters.NE = int(parameters.nE * parameters.N)  # Number of excitatory neurons
parameters.NI = int(parameters.nI * parameters.N)  # Number of inhibitory neuronsparameters.IeE = 1.07
parameters.IeE = -2.5
parameters.IeI = -2.5
parameters.pEE = 1.0
parameters.pEI = 1.0
parameters.pIE = 1.0
parameters.pII = 1.0
parameters.wEE = 8.0
parameters.scale = 0.4
parameters.wIE = parameters.wEE
parameters.wEI = parameters.scale * parameters.wEE
parameters.wII = parameters.scale * parameters.wEE
parameters.tstop = 10
parameters.Nt = int(parameters.tstop / parameters.dt)
parameters.simPhi = 1.0
parameters.Vthres = 1.0

t_start_perturb1 = 1
t_start_perturb2 = 5
t_end_perturb1 = 2
t_end_perturb2 = 6
perturb_amp = 15.0

np.random.seed(894956166)
W = get_weight_matrix_Exc_Inh_N(parameters)
spkTrain, memVol, gSyn, spkTimes, count = sim_network_Perturb(parameters, W, t_start_perturb1, t_start_perturb2, t_end_perturb1, t_end_perturb2, perturb_amp)

t_plot = np.arange(0, parameters.Nt, 1) * parameters.dt

In [6]:
filename = '../Data/Figure 5/300Data_Exp.npz'
data = np.load(filename)

fr_Sim_L = np.flip(data['arr_0'])
fr_Sim_H = np.flip(data['arr_1'])

fr_MF_L = np.flip(data['arr_4'])
fr_MF_H = np.flip(data['arr_5'])

fr_MF_Perturb_L = np.flip(data['arr_8'])
fr_MF_Perturb_H = np.flip(data['arr_9'])

fr_MF_SC_L = np.flip(data['arr_12'])
fr_MF_SC_H = np.flip(data['arr_13'])

In [7]:
filename = '../Data/Figure 5/Data_Renewal_Exp_EI.npz'
data = np.load(filename)

n_Renewal_L = np.flip(data['arr_0'])
n_Renewal_H = np.flip(data['arr_1'])

n_Renewal_L = np.nan_to_num(n_Renewal_L)
n_Renewal_H = np.nan_to_num(n_Renewal_H)

In [8]:
E_plot = np.arange(-4.0, 0.2, 0.25)
E_plot

array([-4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25, -2.  ,
       -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ])

In [9]:
COLOR = (0., 0., 0.)

def color_conv(color_range):
    return (COLOR[0] + color_range, COLOR[1], COLOR[2]) 


fontsize=10
J_limL = 1.0
J_limH = 10.0
g_limL = 0.0
g_limH = 1.0
E_limL = -5.0
E_limH = 4.0

f, ax = plt.subplots(2,2,dpi=600,layout='constrained')

tt = 1 / len(theta)
for i in range(len(theta)):
    if i == 0 or i == len(theta)-1:
        ax[0, 0].plot(g, E_plot_phase0[:,i], color=color_conv(i*tt), linewidth=2, label='$\\theta=$'+str(theta[i]))
    else:
        ax[0, 0].plot(g, E_plot_phase0[:,i], color=color_conv(i*tt), linewidth=2)

    if theta[i] == 1.0:
        ax[0, 0].plot(g, E_plot_phase0[:,i], color=color_conv(i*tt), linewidth=4)
        ax[0, 0].plot(g, E_plot_phase1[:,i], color=color_conv(i*tt), linewidth=4)
    else:
        ax[0, 0].plot(g, E_plot_phase1[:,i], color=color_conv(i*tt), linewidth=2)


ax[0, 0].set_xlabel('J', fontsize=fontsize)
ax[0, 0].set_ylabel('$\\mathcal{E}$', fontsize=fontsize)
ax[0, 0].text(0.4, -4.5, 'B', fontsize=fontsize)
ax[0, 0].text(0.3, 3.0, 'M', fontsize=fontsize)
ax[0, 0].plot(0.4, -2.5, 'X', color='orange', markersize=4.0)
xticks = [0.0, 0.4, 0.8]
yticks = [-5, -0.5, 4.0]
ax[0, 0].set_xticks(xticks, xticks, fontsize=fontsize)
ax[0, 0].set_yticks(yticks, yticks, fontsize=fontsize)
ax[0, 0].set_xlim(g_limL, 0.8)
ax[0, 0].set_ylim(E_limL, E_limH)
ax[0, 0].legend(frameon=False, loc='upper right', fontsize=fontsize)
# ax[0, 0].set_aspect(1./ax[0,0].get_data_ratio())

ax[0, 1].plot(g[idx0[1]], J_var[idx0[0]],'k')
ax[0, 1].plot(g[idx1[1]], J_var[idx1[0]],'k')
ax[0, 1].set_xlabel('g', fontsize=fontsize)
ax[0, 1].set_ylabel('J', fontsize=fontsize)
ax[0, 1].text(0.3, 7.0, 'B', fontsize=fontsize)
ax[0, 1].text(0.1, 9.0, 'M', fontsize=fontsize)
ax[0, 1].text(0.7, 5.0, 'M', fontsize=fontsize)
# ax[0, 1].set_aspect(1./ax[0,1].get_data_ratio())
xticks = [0.0, 0.5, 1.0]
yticks = [1, 5.5, 10]
ax[0, 1].set_xticks(xticks, xticks, fontsize=fontsize)
ax[0, 1].set_yticks(yticks, yticks, fontsize=fontsize)


idx = np.argwhere(spkTimes[:,1] <= parameters.NE)
ax[1, 0].plot(spkTimes[:,0], spkTimes[:,1], '|', markersize=.8)
ax[1, 0].plot(spkTimes[idx,0], spkTimes[idx,1], '|', markersize=.8)
ax[1, 0].set_xlabel('Time (ms/{})'.format(r'$\tau$'), fontsize=fontsize)
ax[1, 0].set_ylabel('Neuron #', fontsize=fontsize)
# ax[1, 0].set_aspect(1./ax[1,0].get_data_ratio())
xticks = [0, 5, 10]
yticks = [0, 50, 100]
ax[1, 0].set_xticks(xticks, xticks, fontsize=fontsize)
ax[1, 0].set_yticks(yticks, yticks, fontsize=fontsize)
inset_ax2 = ax[1, 0].inset_axes([0., 1.05, 1.0, 0.2])
inset_ax2.plot(t_plot, gSyn, 'k', linewidth=1.0)
inset_ax2.set_xticks([])
inset_ax2.set_yticks([]) 
inset_ax2.spines[['left', 'right', 'top', 'bottom']].set_visible(False)


idxE = np.where(np.abs((fr_MF_H - fr_MF_L) < 10e-8) & (fr_MF_H > 0.))[0][0]
ax[1, 1].plot(E_plot, fr_MF_H, '-g', linewidth=2.0)
ax[1, 1].plot(E_plot[0:idxE-1], fr_MF_L[0:idxE-1], '-g', linewidth=2.0, label='MF')
ax[1, 1].plot(E_plot, fr_MF_Perturb_H, '-c', linewidth=2.0)
ax[1, 1].plot(E_plot[0:idxE-1], fr_MF_Perturb_L[0:idxE-1], '-c', linewidth=2.0, label='1 loop')
ax[1, 1].plot(E_plot, n_Renewal_H, '-', color='navy', linewidth=2.0, label='Exact')
ax[1, 1].plot(E_plot[0:idxE-1], n_Renewal_L[0:idxE-1], '-', color='navy', linewidth=2.0)
ax[1, 1].plot(E_plot, fr_Sim_H, 'o', color='brown',markerfacecolor='None')
ax[1, 1].plot(E_plot[0:idxE-1], fr_Sim_L[0:idxE-1], 'o', color='brown',markerfacecolor='None', label='Sim')
ax[1, 1].set_xlabel('$\\mathcal{E}$', fontsize=fontsize)
ax[1, 1].set_ylabel('Firing Rate', fontsize=fontsize)
xticks = [-4.0, -2.0, 0.0]
yticks = [0.0, 16.0, 36.0]
ax[1, 1].set_xticks(xticks, xticks, fontsize=fontsize)
ax[1, 1].set_yticks(yticks, yticks, fontsize=fontsize)
inset_ax3 = ax[1, 1].inset_axes([0.05, 0.45, 0.30, 0.24])
inset_ax3.plot(E_plot[0:idxE-1], fr_Sim_L[0:idxE-1], 'o', color='brown',markerfacecolor='white')
inset_ax3.plot(E_plot[0:idxE-1], fr_MF_L[0:idxE-1], '-g', linewidth=2.0, label='MF')
inset_ax3.plot(E_plot[0:idxE-1], fr_MF_Perturb_L[0:idxE-1], '-c', linewidth=2.0)
inset_ax3.plot(E_plot[0:idxE-1], n_Renewal_L[0:idxE-1], '-', color='navy', linewidth=2.0)
xticks = [-4.0, -2.0]
yticks = [0., 0.12]
inset_ax3.set_xticks(xticks, xticks, fontsize=8)
inset_ax3.set_yticks(yticks, yticks, fontsize=8) 
inset_ax3.yaxis.tick_right()
ax[1, 1].legend(frameon=False, loc=[0.7, 0.4],fontsize=7)
# ax[1, 1].set_aspect(1./ax[1,1].get_data_ratio())

sns.despine(ax=ax[0, 0])
sns.despine(ax=ax[0, 1])
sns.despine(ax=ax[1, 0])
sns.despine(ax=ax[1, 1])

f.savefig('./Figures/Figure5.pdf', format='pdf')
plt.show()



KeyboardInterrupt



Error in callback <function _draw_all_if_interactive at 0x113276c00> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



Error in callback <function flush_figures at 0x1249a6b60> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [ ]:
J = 4.8
fr_MF_SC_L_Resummed = np.zeros((len(E_plot), ))
fr_MF_SC_H_Resummed = np.zeros((len(E_plot), ))
theta = 1.0
dt = 0.001

for i in range(len(E_plot)):
    E1 = E_plot[i]
    V = np.random.uniform(4,6)
    n = np.exp(V - theta)
    for t in range(1000):

        phi = np.exp(V - theta)
        D_vnT2 = 0.5 * V**2 / (1 + n + V * phi)
        
        V += dt * (- V + E1 + (J - V) * n - 0.5 * V**2 * phi**2 / (1 + n + V * phi) )
        n = phi * (1 + 0.5 * phi * (np.cosh(D_vnT2) - 1) )

    fr_MF_SC_H_Resummed[i] = n

        

In [ ]:
f, ax = plt.subplots(1,1,dpi=600,layout='constrained')
idxE = np.where(np.abs((fr_MF_H - fr_MF_L) < 10e-8) & (fr_MF_H > 0.))[0][0]
ax.plot(E_plot, fr_MF_H, '-g', linewidth=2.0)
ax.plot(E_plot[0:idxE-1], fr_MF_L[0:idxE-1], '-g', linewidth=2.0, label='MF')
ax.plot(E_plot, fr_MF_Perturb_H, '-c', linewidth=2.0)
ax.plot(E_plot[0:idxE-1], fr_MF_Perturb_L[0:idxE-1], '-c', linewidth=2.0, label='1 loop')
ax.plot(E_plot, fr_MF_SC_H, '-r', linewidth=2.0)
ax.plot(E_plot[0:idxE-1], fr_MF_SC_L[0:idxE-1], '-r', linewidth=2.0, label='SC 1 loop')
ax.plot(E_plot, fr_MF_SC_H_Resummed, '--r', linewidth=2.0, label='SC 1 loop Resummed')
ax.plot(E_plot, n_Renewal_H, '-', color='navy', linewidth=2.0, label='Exact')
ax.plot(E_plot[0:idxE-1], n_Renewal_L[0:idxE-1], '-', color='navy', linewidth=2.0)
ax.plot(E_plot, fr_Sim_H, 'o', color='brown',markerfacecolor='None')
ax.plot(E_plot[0:idxE-1], fr_Sim_L[0:idxE-1], 'o', color='brown',markerfacecolor='None', label='Sim')
ax.set_xlabel('$\\mathcal{E}$', fontsize=fontsize)
ax.set_ylabel('Firing Rate', fontsize=fontsize)
xticks = [0.0, 0.75, 1.5]
yticks = [0.0, 8.0, 16.0]
ax.set_xticks(xticks, xticks, fontsize=fontsize)
ax.set_yticks(yticks, yticks, fontsize=fontsize)
inset_ax3 = ax.inset_axes([0.02, 0.42, 0.30, 0.25])
inset_ax3.plot(E_plot[0:idxE-1], fr_Sim_L[0:idxE-1], 'o', color='brown',markerfacecolor='white')
inset_ax3.plot(E_plot[0:idxE-1], fr_MF_L[0:idxE-1], '-g', linewidth=2.0, label='MF')
inset_ax3.plot(E_plot[0:idxE-1], fr_MF_Perturb_L[0:idxE-1], '-c', linewidth=2.0)
inset_ax3.plot(E_plot[0:idxE-1], n_Renewal_L[0:idxE-1], '-', color='navy', linewidth=2.0)
xticks = [0., 0.4]
yticks = [0., 0.3]
inset_ax3.set_xticks(xticks, xticks, fontsize=8)
inset_ax3.set_yticks(yticks, yticks, fontsize=8) 
inset_ax3.yaxis.tick_right()
ax.legend(frameon=False, loc=[0.7, 0.35],fontsize=7)

sns.despine(f)